In [6]:
import os

import h5py
import numpy as np
import matplotlib.pyplot as plt


plt.rc("font", family=["Helvetica", "Arial"])
plt.rc("text", usetex=True)
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}\usepackage{amssymb}\usepackage{physics}'
plt.rc("xtick", top=True, direction="in")
plt.rc("ytick", right=True, direction="in")

In [7]:
h5py.File('datasets/1.hdf5').keys()

<KeysViewHDF5 ['Covariances', 'Expected_returns', 'Final_circuit_sample_probabilities', 'Final_circuit_sample_states', 'Max_cost', 'Min_cost', 'Min_cost_state', 'N', 'Normalized_cost', 'Optimizer_maxfev', 'Optimizer_nfev', 'Rng_seed', 'k', 'layers', 'status', 'type']>

In [8]:
results_1, results_2 = {}, {}
for file_name in os.listdir('datasets'):
    if file_name != '.DS_Store':
        with h5py.File('datasets/'+file_name) as f:
            Normalized_cost = f['Normalized_cost'][()]
            if f['type'][()] == 1:
                if f['N'][()] not in list(results_1.keys()):
                    results_1[f['N'][()]] = [Normalized_cost]
                else:
                    results_1[f['N'][()]].append(Normalized_cost)
            if f['type'][()] == 2:
                if f['N'][()] not in list(results_2.keys()):
                    results_2[f['N'][()]] = [Normalized_cost]
                else:
                    results_2[f['N'][()]].append(Normalized_cost)
        
final_results_1, final_results_2 = {}, {}
for layer in np.sort(list(results_1.keys())):
    final_results_1[layer] = (np.mean(results_1[layer]), np.std(results_1[layer]), np.min(results_1[layer]))
for layer in np.sort(list(results_2.keys())):
    final_results_2[layer] = (np.mean(results_2[layer]), np.std(results_2[layer]), np.min(results_2[layer]))

In [9]:
final_results_2

{2: (0.0, 0.0, 0.0, 1.0)}

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,6))
fig.suptitle(r'$k=\lfloor\frac{N}{2}\rfloor, \quad S=100, \quad$ "COBYLA maxiter" = 250',size=15)
plt.subplots_adjust(hspace=0)
# Shared x-values
n_vals = [2 + i for i in range(len(list(final_results_1.keys())))]
err_scaling = 1

# Plottig stuff
ax[0].errorbar(n_vals, [final_results_1[key][0] for key in list(final_results_1.keys())], yerr = [err_scaling*final_results_1[key][1] for key in list(final_results_1.keys())], capsize=3, ls='None', label=f'std. dev.')
ax[0].plot(n_vals,[final_results_1[key][0] for key in list(final_results_1.keys())], 'o', ms=3)
ax[0].plot(n_vals,[final_results_1[key][2] for key in list(final_results_1.keys())], 'x', ms=6, label='best found')

ax[1].errorbar(n_vals, [final_results_2[key][0] for key in list(final_results_2.keys())], yerr = [err_scaling*final_results_2[key][1] for key in list(final_results_2.keys())], capsize=3, ls='None', label=f'std. dev.')
ax[1].plot(n_vals,[final_results_2[key][0] for key in list(final_results_2.keys())], 'o', ms=3)
ax[1].plot(n_vals,[final_results_2[key][2] for key in list(final_results_2.keys())], 'x', ms=6, label='best found')


ax[0].text(2.9,0.1,'Nearest Neighbor (5 layers)',size=14)
ax[1].text(2.05,0.1,r'Nearest Neighbor + Next nearest',size=14)

#_y_ticks_ = [0.0,0.25,0.50,0.75,1.00]
_y_ticks_ = [0.0,0.05,0.10,0.15,0.20]
for _ax_ in range(2):
    ax[_ax_].set_xlim(1.5,max(n_vals)+0.5)
    ax[_ax_].set_ylim(-0.05,0.25)
    ax[_ax_].hlines(0,0.5,max(n_vals)+0.5, ls='--',color='k',lw=1)
    ax[_ax_].hlines(1,0.5,max(n_vals)+0.5, ls='--',color='k',lw=1)
    ax[_ax_].set_yticks(_y_ticks_)

ax[0].set_xticklabels([])
ax[1].set_xticklabels([])
ax[1].text(-0.01,0.25,r'$\frac{|c-c_{\textrm{min}}|}{|c_{\textrm{max}}-c_{\textrm{min}}|}$',rotation=90, size=20)

ax[0].legend(loc=(-0.1,1.05),fontsize=12)

ax[1].set_xticks(n_vals)
ax[1].set_xticklabels([str(n) for n in n_vals], size=14)
ax[1].set_xlabel(r'Nr. stocks', size=16)


plt.savefig('plots/connectivity_strat_varying_N.png', dpi=300)
plt.show()

In [ ]:
L2, N = 5, 15

int(np.floor((L2*(N-1)) / (N-1+N-2)))